# VacationPy

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key as geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output data/mod_6_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chiredzi,-21.0500,31.6667,13.17,58,0,1.54,ZW,1687151899
1,1,geraldton,-28.7667,114.6000,19.23,48,0,2.57,AU,1687151899
2,2,ushuaia,-54.8000,-68.3000,2.81,75,0,3.09,AR,1687151900
3,3,vaini,-21.2000,-175.2000,23.09,73,75,2.06,TO,1687151900
4,4,smithers,54.7804,-127.1743,11.18,87,75,1.03,CA,1687151901


In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500
)


# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
low_wind_city_df = city_data_df[city_data_df['Wind Speed'] < 3.0]

low_wind_city_df

# Drop any rows with null values
low_wind_city_df = low_wind_city_df.dropna(0,how ='any',subset= ['Wind Speed'])

low_wind_city_df

# Display sample data
low_wind_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chiredzi,-21.0500,31.6667,13.17,58,0,1.54,ZW,1687151899
1,1,geraldton,-28.7667,114.6000,19.23,48,0,2.57,AU,1687151899
3,3,vaini,-21.2000,-175.2000,23.09,73,75,2.06,TO,1687151900
4,4,smithers,54.7804,-127.1743,11.18,87,75,1.03,CA,1687151901
5,5,matagami,49.7502,-77.6328,10.06,87,0,1.03,CA,1687151850
6,6,marion,39.7684,-86.1555,22.19,69,100,2.57,US,1687151606
7,7,pueblo nuevo,23.3833,-105.3833,14.94,67,88,1.41,MX,1687151902
8,8,east london,-33.0153,27.9116,10.52,87,0,1.54,ZA,1687151902
9,9,marystown,47.1666,-55.1483,10.48,100,99,2.24,CA,1687151903
10,10,albany,42.6001,-73.9662,14.29,90,43,1.50,US,1687151716


In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = low_wind_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ''


# Display sample data
hotel_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,chiredzi,-21.0500,31.6667,13.17,58,0,1.54,ZW,1687151899,
1,1,geraldton,-28.7667,114.6000,19.23,48,0,2.57,AU,1687151899,
3,3,vaini,-21.2000,-175.2000,23.09,73,75,2.06,TO,1687151900,
4,4,smithers,54.7804,-127.1743,11.18,87,75,1.03,CA,1687151901,
5,5,matagami,49.7502,-77.6328,10.06,87,0,1.03,CA,1687151850,
6,6,marion,39.7684,-86.1555,22.19,69,100,2.57,US,1687151606,
7,7,pueblo nuevo,23.3833,-105.3833,14.94,67,88,1.41,MX,1687151902,
8,8,east london,-33.0153,27.9116,10.52,87,0,1.54,ZA,1687151902,
9,9,marystown,47.1666,-55.1483,10.48,100,99,2.24,CA,1687151903,
10,10,albany,42.6001,-73.9662,14.29,90,43,1.50,US,1687151716,


In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
chiredzi - nearest hotel: Nesbitt Arms Hotel
geraldton - nearest hotel: Ocean Centre Hotel
vaini - nearest hotel: Keleti International Resort
smithers - nearest hotel: Sunshine Inn Hotel
matagami - nearest hotel: Hotel Matagami
marion - nearest hotel: Hilton Garden Inn Indianapolis Downtown
pueblo nuevo - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
marystown - nearest hotel: Marystown Hotel & Convention Centre
albany - nearest hotel: No hotel found
marale - nearest hotel: No hotel found
antigonish - nearest hotel: Microtel Hotel
tura - nearest hotel: No hotel found
abomsa - nearest hotel: Kalid Hotel
edd - nearest hotel: No hotel found
constitucion - nearest hotel: Hostal Alameda
new norfolk - nearest hotel: Woodbridge on the Derwent
portland - nearest hotel: the Hoxton
avarua - nearest hotel: Paradise Inn
vila - nearest hotel: Hotel Miracastro
opuwo - nearest hotel: Ohakane Lodge
mataura - nearest hotel: No hotel found
bluff - neares

ha tinh - nearest hotel: Hong Chau Nha Nhgi
yuci - nearest hotel: 7 Days Inn Jinzhong
nyakahanga - nearest hotel: Karven Hotel
gamba - nearest hotel: No hotel found
puerto leguizamo - nearest hotel: Hotel Girasoles
pozo colorado - nearest hotel: No hotel found
sakakah - nearest hotel: فندق النزل
rosario do sul - nearest hotel: Hotel Ibicuí
qaqortoq - nearest hotel: Hotel Qaqortoq
miraflores - nearest hotel: Hotel Hacienda Panoaya
abha - nearest hotel: SAHAB
alpena - nearest hotel: Ramada of Alpena
maues - nearest hotel: No hotel found
maun - nearest hotel: Center Lodge Conference Center
dawlatabad - nearest hotel: No hotel found
middelburg - nearest hotel: No hotel found
istra - nearest hotel: Воскресенский
hegang - nearest hotel: No hotel found
naze - nearest hotel: ビジネス旅館畠山
bam - nearest hotel: هتل جهانگردی بم
kidal - nearest hotel: No hotel found
ladispoli - nearest hotel: Guest House Roma Express
moron - nearest hotel: La Esquina
rochegda - nearest hotel: No hotel found
bastia - ne

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,chiredzi,-21.0500,31.6667,13.17,58,0,1.54,ZW,1687151899,Nesbitt Arms Hotel
1,1,geraldton,-28.7667,114.6000,19.23,48,0,2.57,AU,1687151899,Ocean Centre Hotel
3,3,vaini,-21.2000,-175.2000,23.09,73,75,2.06,TO,1687151900,Keleti International Resort
4,4,smithers,54.7804,-127.1743,11.18,87,75,1.03,CA,1687151901,Sunshine Inn Hotel
5,5,matagami,49.7502,-77.6328,10.06,87,0,1.03,CA,1687151850,Hotel Matagami


In [9]:
%%capture --no-display

# Configure the map plot with essential attributes
hotel_map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name", "City", "Country", "Lat", "Lng","Wind Speed","Humidity",]
)
# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country,Wind Speed,Humidity)